<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D3_1_Natural_Language.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Procesamiento de Lenguaje Natural
(Natural Language Processing)

En este notebook se muestran ejemplos de como lidiar con datos de texto. A diferencia de las imágenes, un texto no se puede representar numéricamente como la intensidad de una coordenada en una matriz bidimensional. Pero aún podemos hacer una representación (o **codificación**) de un texto usando números. 

TensorFlow nos ofrece varias funcionalidades para ello.

Antes que nada, importamos las bibliotecas necesarias.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokens

En un **corpus** de texto, existe siempre un conjunto finito de palabras. Esto es el **vocabulario** de ese corpus. Una forma muy tradicional de codificar el texto de forma numérica es asignar a cada palabra un **índice** numérico que nos indique la palabra. Para ello, necesitamos una lista ordenada con dicho índice.

Programar esto por nuestra cuenta no parece demasiado complicado, pero hay muchos casos en que hay que tomar decisiones importantes. Por ejemplo:
* Si tenemos memoria limitada y sólo podemos guardar 2000 palabras, ¿cuáles palabras del vocabulario completo debemos ignorar? 
* ¿Qué hacemos con las palabras ignoradas?

Abajo vemos el código que se usa para **tokenizar** un texto (codificar de forma numérica).

In [ ]:
corpus = [
    'había una vez',
    'un gato montes',
    'que tenía la cola al revés',
    'querés que te cuente otra vez?'
]

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 100)
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
print(word_index)

podemos ver que nuestro **vocabulario** es un diccionario que nos dará el índice numérico para cada palabra del corpus. En el caso de arriba, tenemos menos palabras en el corpus de las configuradas en el *tokenizer*. 

Una vez identificado el vocabulario, podemos **codificar** nuestro corpus en secuencias numéricas.

In [ ]:
secuencias = tokenizer.texts_to_sequences(corpus)
print(secuencias)

A partir de la secuencia, desde luego podemos volver al texto original.

In [ ]:
texto = tokenizer.sequences_to_texts(secuencias)
print(texto)

Note como hemos perdido el signo de puntuación final. Esto se debe a que entre los parámetros del [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) hay un filtro, que nos permite controlar qué simbolos se ignoran.

# Valores fuera del vocabulario

Hay muchos motivos por los cuales al analizar un texto pueda aparecer una palabra que no se encuentra en nuestro vocabulario:

* Nuestro corpus no era demasiado variado en cuanto a palabras se refiere.
* La nueva palabra está incorrectamente escrita.
* La nueva palabra está en otro idioma.

Para lidiar con estos casos, podemos usar un parámetro del Tokenizer que nos permite definir un `str` para los casos fuera de vocabulario (out of vocabulary).

In [ ]:
# crear un tokenizer, limitando la cantidad de palabras
tokenizer_oov = tf.keras.preprocessing.text.Tokenizer(num_words = 100, oov_token="<???>")

tokenizer_oov.fit_on_texts(corpus)
word_index = tokenizer_oov.word_index
print(word_index)

Note como el valor que definimos se usa como primer elemento del diccionario.

Veamos que pasa cuando intentamos codificar una oración con una palabra afuera del diccionario.

In [ ]:
# agregamos una palabra al corpus original
corpus_oov = corpus + ['Oración con palábras nunca antes vistas']

# usamos el tokenizador para obtener las secuencias
secuencias_oov = tokenizer_oov.texts_to_sequences(corpus_oov)
print(secuencias_oov)

Observamos como la última secuencia esta formada solo de palábras que no existen en nuestro vocabulario. Claramente, al intentar recuperar el texto, solo podemos hacerlo parcialmente

In [ ]:
texto_oov = tokenizer_oov.sequences_to_texts(secuencias_oov)
print(texto_oov)

In [ ]:
tokenizer.texts_to_sequences(corpus_oov)

# Estandarizar las secuencias

Un problema a la hora de usar oraciones es que su longitud es variable, veamos la longitud de cada oración de nuestro corpus

In [ ]:
for oracion in corpus:
    print(len(oracion.split(' ')))

Una de las secuencias es más larga que las demás. Para lidiar con este problma, podemos seguir varias estrategias, pero por lo general se decide hacer "padding". Esto funciona en dos posibles modos:

* **pre**: va a agregar ceros ANTES de las secuencias cortas
* **post**: va a agregar ceros DESPUES de las secuencias cortas


In [ ]:
padded = pad_sequences(secuencias, padding='pre')
print(padded)

Veamos que texto es equivalente

In [ ]:
tokenizer.sequences_to_texts(padded)

En muchas ocasiones, puede ocurrir que la oración más larga es mucho más larga que lo razonable para nuestro modelo, y las secuencias generadas serían mayoritariamente cero. la funcíon `pad_sequences` soporta un parámetro `maxlen`, que nos sirve de ayuda en estos casos, limitando la longitud de la salida. Para controlar como se cmporta este parámetro (si va a eliminar las palabras desde el comienzo o el final de la oración, podemos definir el parámetro `truncating`). Siempre es bueno tener la  [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) a mano para entender lo que la función hace.

In [ ]:
padded = pad_sequences(secuencias, padding='pre', maxlen=4)
print(padded)

Se debe estudiar en que casos es preferible esto en vez de eliminar la oración del dataset. Ya que se pierde gran parte del texto original.

In [ ]:
tokenizer.sequences_to_texts(padded)

# Créditos 

Este notebook utiliza y modifica contenido del curso online [TensorFlow in Practice](https://www.deeplearning.ai/tensorflow-in-practice/).